In [2]:
import tensorflow as tf

# Prepare data

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('fashion', one_hot=True)
#mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting fashion/train-images-idx3-ubyte.gz
Extracting fashion/train-labels-idx1-ubyte.gz
Extracting fashion/t10k-images-idx3-ubyte.gz
Extracting fashion/t10k-labels-idx1-ubyte.gz


# Build a convolution neural network

we start by building some wrapper functions:
* weight
* bias
* conv2d
* maxpool

In [4]:
def weight_variable(shape):
    init = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init)

def bias(shape):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [5]:
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
x_image = tf.reshape(x, [-1, 28, 28, 1])

# First conv layer

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias([32])

h_conv1 = tf.add(tf.nn.relu(conv2d(x_image, W_conv1)), b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second conv layer

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias([64])

h_conv2 = tf.add(tf.nn.relu(conv2d(h_pool1, W_conv2)), b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


In [6]:
# fully connected layer
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)

In [7]:
# dropout layer
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [10]:
# output layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2)+b_fc2

In [11]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_conv, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i%1000 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
            print('step {}, accuracy: {:.2f}'.format(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})
    
    test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels,
                                            keep_prob: 1.0})
    print('test accuracy: {:.2f}'.format(test_accuracy*100))

step 0, accuracy: 0.16
step 1000, accuracy: 0.84
step 2000, accuracy: 0.88
step 3000, accuracy: 0.90
step 4000, accuracy: 0.86
step 5000, accuracy: 0.90
step 6000, accuracy: 0.94
step 7000, accuracy: 0.82
step 8000, accuracy: 0.86
step 9000, accuracy: 0.96
step 10000, accuracy: 0.90
step 11000, accuracy: 0.90
step 12000, accuracy: 0.96
step 13000, accuracy: 0.92
step 14000, accuracy: 0.90
step 15000, accuracy: 0.98
step 16000, accuracy: 0.92
step 17000, accuracy: 0.90
step 18000, accuracy: 0.96
step 19000, accuracy: 0.94
test accuracy: 90.74
